In [1]:
import pandas as pd
import glob
from read_csv_gz import read_csv_gz
import matplotlib.pyplot as plt
import seaborn as sns

In [58]:
ed_diagnosis_df = read_csv_gz("ed_data/diagnosis.csv.gz")
ed_edstays_df = read_csv_gz("ed_data/edstays.csv.gz")
ed_medrecon_df = read_csv_gz("ed_data/medrecon.csv.gz")
ed_pyxis_df = read_csv_gz("ed_data/pyxis.csv.gz")
ed_triage_df = read_csv_gz("ed_data/triage.csv.gz")
ed_vitalsign_df = read_csv_gz("ed_data/vitalsign.csv.gz")
hosp_admissions_df = read_csv_gz("hosp_data/admissions.csv.gz")
hosp_patients_df = read_csv_gz("hosp_data/patients.csv.gz")

In [71]:
ed_edstays_df.shape[0]

425087

In [59]:
hosp_admissions_df.head(5)

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P49AFC,TRANSFER FROM HOSPITAL,HOME,Medicaid,English,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P784FA,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P19UTS,EMERGENCY ROOM,HOSPICE,Medicaid,English,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P06OTX,EMERGENCY ROOM,HOME,Medicaid,English,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P39NWO,EMERGENCY ROOM,NaN,NaN,English,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [60]:
hosp_patients_df.head(5)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000058,F,33,2168,2020 - 2022,NaN
3,10000068,F,19,2160,2008 - 2010,NaN
4,10000084,M,72,2160,2017 - 2019,2161-02-13


In [114]:
def merge_df(df_chef, df_2, df_3):
    # df_2: age, df_3: insurance, marital status, language
    df_2_cleaned = df_2[['subject_id', 'anchor_age']].drop_duplicates()
    # df_3_cleaned = df_3[['subject_id', 'insurance', 'marital_status', 'language']].drop_duplicates(subset='subject_id')
    df_3_cleaned = df_3[['subject_id', 'language']].drop_duplicates()
    print(len(df_3_cleaned['subject_id'].unique()), len(df_3_cleaned))
    ans = pd.merge(df_chef, df_3_cleaned, on='subject_id', how='left')
    # print(ans.shape[0])
    # print(ans.head(5))
    ans = pd.merge(ans, df_2_cleaned, on='subject_id', how='left')
    # print(ans.shape[0])
    return ans


In [115]:
merged_df = merge_df(df_chef=ed_edstays_df, df_2=hosp_patients_df, df_3=hosp_admissions_df)

223452 223452


In [95]:
merged_df.shape[0]

425087

In [85]:
merged_df[merged_df['subject_id'] == 10000032]

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,anchor_age
0,10000032,22595853.0,33258284,2180-05-06 19:17:00,2180-05-06 23:30:00,F,WHITE,AMBULANCE,ADMITTED,52.0
1,10000032,22841357.0,38112554,2180-06-26 15:54:00,2180-06-26 21:31:00,F,WHITE,AMBULANCE,ADMITTED,52.0
2,10000032,25742920.0,35968195,2180-08-05 20:58:00,2180-08-06 01:44:00,F,WHITE,AMBULANCE,ADMITTED,52.0
3,10000032,29079034.0,32952584,2180-07-22 16:24:00,2180-07-23 05:54:00,F,WHITE,AMBULANCE,HOME,52.0
4,10000032,29079034.0,39399961,2180-07-23 05:54:00,2180-07-23 14:00:00,F,WHITE,AMBULANCE,ADMITTED,52.0


In [118]:
cols = ['anchor_age']
for col in cols:
    print(col, merged_df[col].isnull().sum() / merged_df.shape[0])

anchor_age 0.00017878693067536764


def merge(df_chef, df_2, df_3):
    # df_1 contains subject_id, df_2 contains age, df_3 contains insurance, languge, 